In [2]:
import re, os
import pandas as pd
import numpy as np
import time

import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point

import matplotlib.pyplot as plt
%matplotlib inline

subfolder = 'fringe250smth7' ## areas to load
data_path = 'C:/Users/SpiffyApple/Documents/USC/Courses/Fall2018/JorgeWinterWork/LandScan Global 2015/LandScan Global 2015/lspop2015'

# Introduction

In this notebook, I do two things: 

    a. I geocode Jorge's set of households to the metro areas that I found using LandScan data. 
    
    b. I compute the exposure rate for each metropolitan area. 
    
All data used in here come from Landscan work_ver4.R including:

    - peruAreas is a point-level data of population and size of each found metro area
    
    - Polygon for each metro area
    
    - LandScan data as polygons 

## Geocode survey of households

I first load the peruAreas dataset that contains the population, size, and identity for each found metropolitan area so I can do a spatial join on the landscan-found metropolitan area polygons. To geocode Jorge's surveys, I do a spatial join between the metro area polygons and the survey data.

In [85]:
## load Peru Areas in their full so I can identify polygons
df = pd.read_csv("/".join([data_path, subfolder,'peruAreas_.csv']))
df.drop("Unnamed: 0",inplace=True,axis=1)
peruAreas = gpd.GeoDataFrame(
    df.drop(['x', 'y'], axis=1),
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(df.x, df.y)])

,zone,sqrkm,pop,x,y,areaType
0,1,32,60897.836735,-80.445783,-3.565858,core
1,2,103,408802.204082,-73.260778,-3.749409,core
2,3,35,76211.551020,-80.682040,-4.897590,core
3,4,89,397168.755102,-80.635240,-5.192622,core
4,6,45,106272.489796,-76.362008,-6.485543,core


In [3]:
## Load households
#point = gpd.GeoDataFrame.from_file("/".join([data_path,'households','households.shp'])) # or geojson etc

## I load the STATA data directly instead of putting it through GIS software
tmpPath = 'C:/Users/SpiffyApple/Documents/USC/Courses/Fall2018/JorgeWinterWork'
hhs= pd.read_stata("/".join([tmpPath, 'unique_ids.dta']))

point = gpd.GeoDataFrame(
    hhs.drop(['longitude', 'latitude'], axis=1),
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(hhs.longitude, hhs.latitude)])

In [97]:
## load identified metro areas
polyDict = {}
for areaType in ['core','coreFringe','settlements']:
    print("Loading: %s" %areaType)
    polyDict[areaType] = gpd.GeoDataFrame.from_file("/".join([data_path,subfolder,areaType,areaType+'.shp']))
    print("Shapefiles loaded")

Loading: core
Shapefiles loaded
Loading: coreFringe
Shapefiles loaded
Loading: settlements
Shapefiles loaded


In [98]:
peruAreas.areaType.unique()

array(['core', 'fringe', 'core&fringe', 'settlement'], dtype=object)

In [99]:
polyDict['core'] = sjoin(polyDict['core'], peruAreas[peruAreas.areaType=='core'],how='left').drop('index_right',axis=1)
polyDict['coreFringe'] = sjoin(polyDict['coreFringe'], peruAreas[peruAreas.areaType=='core&fringe'],how='left').drop('index_right',axis=1)
polyDict['settlements'] = sjoin(polyDict['settlements'], peruAreas[peruAreas.areaType=='settlement'],how='left').drop('index_right',axis=1)

In [101]:
pointDict = {}

for areaType in ['core','coreFringe','settlements']:
    print("Performing spatial join with %s" %areaType)
    start_time = time.time()
    pointDict[areaType] = sjoin(point, polyDict[areaType], how='inner')
    pointDict[areaType].dropna(subset=['areaType'],inplace=True) ## drop unmatched households
    
    print("\tSpatial join done. It took: %.2fseconds" %(time.time()-start_time))

Performing spatial join with core
	Spatial join done. It took: 73.19seconds
Performing spatial join with coreFringe
	Spatial join done. It took: 69.13seconds
Performing spatial join with settlements
	Spatial join done. It took: 81.45seconds


In [104]:
households = pd.concat(pointDict, keys=None, sort=True).drop_duplicates(subset=['field_1'])
households.shape

(608491, 16)

In [105]:
print("Number of observations in each area:")
households.groupby('areaType').field_1.count()

Number of observations in each area:


areaType
core           418096
core&fringe     52395
settlement     138000
Name: field_1, dtype: int64

In [107]:
superfluousCols = ['index_right','geometry','layer','longitude','latitude']
households.drop(superfluousCols, axis=1, inplace=True)
households.reset_index(drop=True,inplace=True)

In [109]:
# save households to a csv
households.to_csv("/".join([data_path, 'fringe250smth7','geoCodedhouseholds.csv']))

## Compute Exposure Rate

For each square, I find its share of the population within its metro area. I then create half-mile buffers around each square, load the smoothed landscan data of Peru, and find all of the landscan squares that lie within each buffer. To conclude, I sum the population within the landscan squares for each buffer, multiply the total by the square's share, and then sum the products over each metropolitan area. 

In [146]:
## load shapefiles:
squareDict = {}
for areaType in ['coreFringesLandScan','settlementsLandScan']:
    s = re.sub("LandScan",'',areaType)
    print("Working on: %s" %s)
    squareDict[s] = gpd.GeoDataFrame.from_file("/".join([data_path,subfolder,areaType, s+'.shp']))   

Working on: coreFringes
Working on: settlements


In [147]:
## geocode the squares
areaType = 'coreFringes'
squareDict[areaType] = sjoin(squareDict[areaType],polyDict['coreFringe'],  how='left')
squareDict[areaType].drop(['index_right','sqrkm', 'layer_right'],axis=1,inplace=True)
squareDict[areaType].rename(columns = {'layer_left':'sqrPop'},inplace=True)
squareDict[areaType]['share'] = squareDict[areaType].sqrPop/squareDict[areaType]['pop']

areaType = 'settlements'
squareDict[areaType] =sjoin(squareDict[areaType],polyDict[areaType],how='left')
squareDict[areaType].drop(['index_right','sqrkm','layer_right'],axis=1,inplace=True)
squareDict[areaType].rename(columns = {'layer_left':'sqrPop'},inplace=True)
squareDict[areaType]['share'] = squareDict[areaType].sqrPop/squareDict[areaType]['pop']

In [148]:
## load peru landscan
landscan = gpd.GeoDataFrame.from_file("/".join([data_path,'perulandscan','peru.shp']))

In [149]:
## create buffers and intersect with Peru
bufferDict = {}
inDict = {}

for areaType in ['coreFringes','settlements']:
    print("Working on: %s" %areaType)
    bufferDict[areaType] = squareDict[areaType]
    bufferDict[areaType]['geometry'] = squareDict[areaType].geometry.buffer(.0072)
    inDict[areaType] = sjoin(bufferDict[areaType],landscan, how='left') # spatial join buffers and landscan

Working on: coreFringes
Working on: settlements


In [150]:
## compute total Landscan population in the buffer
sumDict = {}
for areaType in ['coreFringes','settlements']:
    sumDict[areaType] = pd.concat([squareDict[areaType], inDict[areaType].groupby(axis=0, level=0).sqrPop.agg(['sum'])],axis=1)

In [152]:
## rename sum column to something more understandable
sumDict['coreFringes'].rename(columns = {'sum':'totalExposure'},inplace=True)
sumDict['settlements'].rename(columns = {'sum':'totalExposure'},inplace=True)

In [153]:
## multiply shares by total population in circle
for areaType in ['coreFringes','settlements']:
    print("working on %s" %areaType)
    sumDict[areaType].loc[:,'exposure'] = sumDict[areaType].share*sumDict[areaType].loc[:,'totalExposure']

working on coreFringes
working on settlements


In [159]:
## sum over squares in circles to compute exposure 
exposureDict = {}
for areaType in ['coreFringes','settlements']:
    exposureDict[areaType] = sumDict[areaType].groupby('zone')[['exposure','share']].sum()
    
## process output and save to a csv    
exposureDF = pd.concat(exposureDict).reset_index(level=0)
subset = (peruAreas.areaType=='core&fringe') | (peruAreas.areaType=='settlement')
exposureDF =exposureDF.merge(peruAreas.loc[subset].drop('geometry',axis=1),left_index=True, right_on='zone')
exposureDF.to_csv("/".join([data_path, subfolder, 'exposures.csv']))